# Implementação de Referência

Esta é uma implementação de referência das interfaces `IEnquirer` e `IResponder` que serão usadas no trabalho.

Ela também está montada sobre a lógica de componentes de software apresentada no notebook que está no diretório `s04components`.

Nessa implementação é criado um componente paciente `Patient` que sorteia uma das doenças e um componente doutor `CrazyDoctor` que tentará descobrir a doença.

Como o doutor é louco, ele apenas sorteia uma das doenças também. Apesar de fazer perguntas ao paciente, ele ignora as respostas e tenta advinhar a doença. Na maioria das vezes ele erra. Mas, lembre-se, essa não é a solução para o problema, é apenas uma implementação mostrando como se usa as interfaces e a lógica de componentes.

## Interfaces relacionadas ao componente `DataSet`

In [1]:
public interface IDataSource {
  public String getDataSource();
  public void setDataSource(String dataSource);
}

public interface ITableProducer {
  String[] requestAttributes();
  String[][] requestInstances();
}

public interface ITableProducerReceptacle {
  public void connect(ITableProducer producer);
}

## Componente `DataSet`

In [2]:
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.util.ArrayList;

public interface IDataSet extends IDataSource, ITableProducer {
}

public class DataSetComponent implements IDataSet {
  private String dataSource = null;
  private String[] attributes = null;
  private String[][] instances = null;
  
  public DataSetComponent() {
    /* nothing */
  }

  public String getDataSource() {
    return dataSource;
  }

  public void setDataSource(String dataSource) {
    this.dataSource = dataSource;
    if (dataSource == null) {
      attributes = null;
      instances = null;
    } else
      readDS();
  }
  
  public String[] requestAttributes() {
    return attributes;
  }
  
  public String[][] requestInstances() {
    return instances;
  }
  
  private void readDS() {
    ArrayList<String[]> instArray = new ArrayList<String[]>();
    try {
      BufferedReader file = new BufferedReader(new FileReader(dataSource));
        
      String line = file.readLine();
      if (line != null) {
        attributes = line.split(",");
        line = file.readLine();
        while (line != null) {
          String[] instLine = line.split(",");
          instArray.add(instLine);
          line = file.readLine();
        }
        instances = instArray.toArray(new String[0][]);
      }
        
      file.close();
    } catch (IOException erro) {
      erro.printStackTrace();
    }
  }
  
}

## Interfaces de Referência

Estas são as três interfaces de referência `IResponder`, `IResponderReceptacle` e `IEnquirer`, que deverão ser usadas na sua solução. Elas não podem ser modificadas para a **Parte 1** do trabalho.

In [3]:
public interface IResponder {
    public String ask(String question);
    public boolean finalAnswer(String answer);
}

public interface IResponderReceptacle {
    public void connect(IResponder responder);
}

public interface IEnquirer {
    public void startInterview();
}

## Componente `Patient`

Implementação de referência de um componente `Patient` que é ligado ao componente `DataSet` (pela interface `ITableProducerReceptacle`) e implementa a interface `IResponder`. A interface `IPatient` foi criada para juntar a interface provida (`IResponder`) e a interface requerida (`ITableProducerReceptacle`).

In [4]:
import java.util.Random;

// combinando duas interfaces para IPatient
public interface IPatient extends IResponder, ITableProducerReceptacle {
}

public class Patient implements IPatient {
    private int patientN = 0;
    
    private ITableProducer producer;
    
    private String attributes[];
    private String patientInstance[];
    
    public void connect(ITableProducer producer) {
        this.producer = producer;

        attributes = producer.requestAttributes();
        String instances[][] = producer.requestInstances();

        patientN = (int)(Math.random() * instances.length);
        patientInstance = instances[patientN];
        
        System.out.println("Patient selected: " + patientN);
        System.out.println("Patient's disease: " + patientInstance[attributes.length - 1]);
    }
    
    public String ask(String question) {
        String result = "unknown";
        
        for (int a = 0; a < attributes.length - 1; a++)
            if (question.equalsIgnoreCase(attributes[a]))
                result = (patientInstance[a].equals("t")) ? "yes" : "no";
        
        return result;
    }

    public boolean finalAnswer(String answer) {
        boolean result = false;
        if (answer.equalsIgnoreCase(patientInstance[attributes.length - 1]))
            result = true;
        return result;
    }
}

## Componente `CrazyDoctor`

Implementação de um médico louco. Este médico não faz o diagnóstico correto na maioria das vezes, mas usa as interfaces corretamente. Ele serve como referência para o uso das interfaces e componentes.

O componente `CrazyDoctor` que é ligado ao componente `DataSet` (pela interface `ITableProducerReceptacle`), é ligado ao componente `Patient` (pela interface `IResponderReceptacle`) e implementa a interface `IEnquirer`. A interface `IDoctor` foi criada para juntar a interface provida (`IEnquirer`) e as interfaces requeridas (`IResponderReceptacle` e `ITableProducerReceptacle`).

In [5]:
import java.util.Random;

// combinando duas interfaces para IPatient
public interface IDoctor extends IEnquirer, IResponderReceptacle, ITableProducerReceptacle {
}

public class CrazyDoctor implements IDoctor {
    private int patientN = 0;

    private ITableProducer producer;
    private IResponder responder;
  
    public void connect(ITableProducer producer) {
        this.producer = producer;
    }
    
    public void connect(IResponder responder) {
        this.responder = responder;
    }
        
    public void startInterview() {
        String attributes[] = producer.requestAttributes();
        String instances[][] = producer.requestInstances();
        String resultado[] = new String[attributes.length-1];
        boolean linhaTrue = false;
        
        
        for (int a = 0; a < attributes.length - 1; a++){
            System.out.println("Question: " + responder.ask(attributes[a]));
            if("no".equals(responder.ask(attributes[a])))
                resultado[a] = "f";
            else
                resultado[a] = "t";
        }
        
        for(int i = 0; i < attributes.length-1; i++)
            System.out.print(resultado[i]);
            
        for(int a = 0; a < instances.length; a++){
            linhaTrue = true;
            for(int b = 0; b < attributes.length-1; b++){
                if(!resultado[b].equals(instances[a][b]))
                    linhaTrue = false;
            }
            if(linhaTrue){
                /* Se der match, dá o diagnostico e para - preferência ao primeiro diagnóstico */
                System.out.println(a);
                patientN = a;
                break;
            }
        }
        
        
        System.out.println("Disease guess: " + instances[patientN][attributes.length - 1]);
        boolean result = responder.finalAnswer(instances[patientN][attributes.length - 1]); 
        System.out.println("Result: " + ((result) ? "I am right =)" : "I am wrong =("));
    }
    
}

In [24]:
// instanciando o componente DataSet
IDataSet dataset = new DataSetComponent();
dataset.setDataSource("../../../db/zombie/zombie-health-spreadsheet-ml-training.csv");

// instanciando o componente paciente
IPatient aPatient = new Patient();

// conectando-o no componente DataSet
aPatient.connect(dataset);

// instanciando o componente doutor louco
IDoctor cDoctor = new CrazyDoctor();

// conectando-o ao componente DataSet
cDoctor.connect(dataset);

// conectando-o ao componente paciente
cDoctor.connect(aPatient);

// executando a entrevista
cDoctor.startInterview();

 /* Equipiada - Andreis Purim, Pedro Pupo, Marcos Diaz, Guilherme Ramirez */

public interface IStatistic extends ITableProducerReceptacle{
    public void connect(ITableProducer producer);
    // Conecta ao produtor de tabela;
    public String[][] findUnique();
    // Encontra doenças unicas na tabela, retorna doenças e frequencias
    public String[][] frequencyTableAttributes();
    // Encontra frequencia de atributos por doença;
    public String[][] frequencyTableDisease();
    // Encontra frequencia de doenças por atributo
    public String[][] percentageAttributes();
    // Porcentagem das frequencias
    public String[][] percentageDiseases();
    // Porcentagem dos atributos
    public String[][] likely(String[] symptoms);
    // Checa probabilidade dados os sintomas;
}


// Interface para o uso de Principal Component Analysis para reducao da dimensao dos dados do ZombieHealth, de modo a permitir melhor visualizacao deles. 
// O principal objetivo de proporcionar melhor visualizacao eh facilitar a busca por padroes entre as doencas.

public interface PCA_Analysis extends IDataSet, ITableProducerReceptacle{
    //retorna um vetor com a variacao dos componentes principais em ordem decrescente
    public float[] principal_components (int nDimensions);
    //nDimesnions = 1: mostra a distribuicao das doencas ao longo do componente de maior variacao
    //nDimensions = 2: mostra um plano com a distribuicao das doencas ao longo dois dois componentes de maoir variacao
    public void visualize (int nDimensions);
}

public interface IDebugger{
    public void connect(IDoctor doc, IPatient pat);
    // Conecta ao medico e ao doutor
    public String[] debugPatient();
    // Retorna string de debug do paciente;
    public String[] debugDoctor();
    // Retorna string de debug do doutor;
    public String[] debugDiagnosis();
    // Retorna string de debug da diagnose;
}

// Usar interface JavaPoly para fazer interface gráfica.
/* 
<!-- Include the Polyfill -->
<script src="https://www.javapoly.com/javapoly.js"></script>

<!-- Include your favorite Java libraries (jar files) -->
<script type="text/java" src="http://www.yourdomain.com/jimboxutilities.jar"></script>
<script type="text/java" src="http://www.yourdomain.com/guava.jar"></script>
<script type="text/java" src="http://www.yourdomain.com/apache-commons.jar"></script>

<!-- Or, include individual .class files -->
<script type="text/java" src="http://www.yourdomain.com/com/yourpackage/Foo.class"></script>
<script type="text/java" src="http://www.yourdomain.com/com/yourpackage/Noise.class"></script>

<!-- Or just include the source directly -->
<script type="text/java" src="http://www.yourdomain.com/com/yourpackage/Bar.java"></script>
<script type="text/java" src="http://www.yourdomain.com/com/yourpackage/Girls.java"></script> 
*/


public interface IRealistic_Dialogue{
    public void additional(boolean additional);
    // Adiciona dialogos adicionais
    public void virose(boolean virose);
    // Todas as doenças se tornaram viroses
    public void identifiers(boolean additional);
    // Dialogos como teatro
    public String questions(String question, boolean print);
    // Varia as perguntas
    public String answers(String answer, boolean print);
    // Varia as respostas
    
}


/***************************************** Classe Estatística *****************************************/
public class Statistic {
    private ITableProducer producer;
    private String attributes[];
    private String instances[][];
    private String unique[];
    private int frequency[][];
    private int counter[];

    // Função de Conectar com o produtor da tabela;
    public void connect(ITableProducer producer){
        this.producer = producer;
    }
    /******************************* Acha Unicos *******************************/
    // Irá econtrar as doenças únicas e suas frequências
    public void findUnique(){ 
        int size = producer.requestInstances().length; 
        attributes = producer.requestAttributes();
        instances = producer.requestInstances();
        String ordenado[] = new String[size];
        
        // Pega todas as strings do final, e depois acha as únicas;
        for(int a = 0; a < size; a++){
            ordenado[a] = instances[a][attributes.length - 1];
        }
        unique = Arrays.stream(ordenado).distinct().toArray(String[]::new);
        
        // Agora faz um contador para ver quantas vezes foi
        // Sim, isso não é efetivo ainda, está cheio de fors
        counter = new int[unique.length+1];
        for(int a = 0; a < counter.length; a++)
            counter[a] = 0;
        for(int a = 0; a < unique.length; a++){
            for(int b = 0; b < instances.length; b++){
                if(unique[a].equals(instances[b][attributes.length - 1])){
                    counter[0]++;
                    counter[a+1]++;
                }
            }
        }
        
        // Junta os dois em uma matriz uniques
        String uniques[][] = new String[unique.length+1][2];
        uniques[0][0] = "total";
        for(int a = 0; a < unique.length; a++)
            uniques[a+1][0] = unique[a];
        for(int a = 0; a < counter.length; a++)
            uniques[a][1] = Integer.toString(counter[a]);
        
        System.out.println("========= Unique Diseases =========");
        for(int a = 0; a < uniques.length; a++){
            System.out.println(uniques[a][0] + " " + uniques[a][1]);
        }
    }
    
    /******************************* Printa Tabela *******************************/
    // Faz a tabela ficar mais bonita de ver. Só isso.
    public void printTable(){
        System.out.println("========= Disease Table =========");
        for (int a = 0; a < instances.length; a++){
            for(int b = 0; b < attributes.length; b++){
                if(!instances[a][b].equals("f"))
                    System.out.print(instances[a][b] + " ");
                else
                    System.out.print("  ");
            }
        System.out.println("");
        } 
    }
    
    /******************************* Acha frequências *******************************/
    public void findFrequencies(){
        frequency  = new int[unique.length+1][attributes.length-1];
        
        // Zera tudo
        for(int a = 0; a < (unique.length+1); a ++){
            for(int b = 0; b < attributes.length-1; b++){
                frequency[a][b] = 0;
            }
        }
        
        // Para cada doença no unique, procura na tabela, se achar, incrementa na matriz de frequencia;
        for(int a = 0; a < unique.length; a ++){
           for(int b = 0; b < instances.length; b++){
               if(instances[b][attributes.length-1].equals(unique[a])){
                   for(int c = 0; c < attributes.length-1; c++){
                        if(instances[b][c].equals("t")){
                            frequency[0][c]++;
                            frequency[a+1][c]++;
                       }
                   }
               }
           } 
        }
        
        // Printa frequencias
        System.out.println("========= Frequencies =========");
        for(int a = 0; a < (unique.length+1); a ++){
            for(int b = 0; b < attributes.length-1; b++){
                System.out.printf("%3d ",frequency[a][b]);
            }
            if(a > 0)
                System.out.print(unique[a-1]);
            else
                System.out.print("total");
            System.out.println("");
        }
    }
    /******************************* Acha porcentagem *******************************/
    public void findPercentage(){
        System.out.println("========= Percentage =========");
        
        // Basicamente percorre nossas frequências e divide, formatação diferente de zeros.
        String percentage[][] = new String[unique.length][attributes.length];
        
        for(int a = 1; a < (unique.length+1); a ++){
            int b;
            for(b = 0; b < attributes.length-1; b++){
                percentage[a-1][b] = String.format("%.2f",(float)frequency[a][b]/frequency[0][b]);
                if(!percentage[a-1][b].equals("0.00"))
                    System.out.print(percentage[a-1][b] + " ");
                else
                    System.out.print("0    ");
            }
            percentage[a-1][b] = unique[a-1];
            System.out.println(percentage[a-1][b]);
        }
    }
    
    public void staticDiagnose(String sintomas){
        float percentageB[][] = new float[frequency.length][(frequency[0]).length];
        System.out.println("");
        
        for(int a = 0; a < frequency.length; a++){
            for(int b = 0; b < frequency[0].length; b++){
                    percentageB[a][b] = (float)frequency[a][b]/counter[0];
                    System.out.print(String.format("%.2f ", percentageB[a][b]));
            }
            if(a == 0)
                System.out.print("\n");
            System.out.println(""); 
        }
        System.out.println("");
        
        int i; 
    } 

}
    

Statistic s = new Statistic();
s.connect(dataset);
s.findUnique();
s.printTable();
s.findFrequencies();
s.findPercentage();
s.staticDiagnose("yellow_tong");

/*for (int a = 0; a < dataset.requestInstances().length; a++){
    for(int b = 0; b < dataset.requestAttributes().length; b++){
        System.out.print(ordenado[a][b] + " ");
    }
    System.out.println("");
}*/

Patient selected: 18
Patient's disease: bite_deficit
Question: no
Question: no
Question: no
Question: no
Question: yes
Question: no
Question: no
fffftff7
Disease guess: bite_deficit
Result: I am right =)
========= Unique Diseases =========
total 19
bacterial_infection 5
bite_deficit 7
viral_infection 3
fights 3
nothing 1
========= Disease Table =========
t t           bacterial_infection 
  t           bacterial_infection 
  t     t   t bite_deficit 
  t t   t     bite_deficit 
    t t       viral_infection 
    t     t   fights 
          t   nothing 
        t     bite_deficit 
  t   t       bacterial_infection 
      t       viral_infection 
  t t       t bite_deficit 
t t           bacterial_infection 
      t     t viral_infection 
    t         fights 
  t     t   t bite_deficit 
  t t   t     bite_deficit 
      t       fights 
  t           bacterial_infection 
        t     bite_deficit 
========= Frequencies =========
  2  10   6   5   6   2   4 total
  2   5   0   1   0   0 